# Therapy Chatbot

The dataset contains 80 user responses, in the response_text column, to a therapy chatbot. Bot said: 'Describe a time when you have acted as a resource for someone else'. User responded. If a response is 'not flagged', the user can continue talking to the bot. If it is 'flagged', the user is referred to help. We are going to predict if it is flagged or not according to users responses.

### Libraries and Utilities